In [ ]:
# | default_exp create_stand_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [2]:
# | export
import os
import collections
import pandas as pd
import pandera as pa
from typing import Dict
from pathlib import Path
from pandera.typing import Series
from pydantic_extra_types.coordinate import Coordinate

In [160]:
# | export

# This class was created for validating the input dataframe
# If the data don't follow the structure specified the function will fail
class StandFile(pa.SchemaModel):
    "Schema for validating the input stand parameter csv file"

    plot_id:Series[str] = pa.Field(description="Plot id from which the data was collected")
    lai_max: Series[float] = pa.Field(ge = 0, description="maximum leaf area index of the stand (m2/m2)")
    latitude: Series[float] = pa.Field(description="latitude of the stand")
    longitude: Series[float] = pa.Field(description="longitude of the stand")

    # Added for making sure that it only accepts the columns specified above
    class Config:
        strict = True


def create_stand_parameters(file_path: Path = None,  # Path to a csv file containing lai_max, latitude and longitude values i.e path/to/parameter_values.csv
                            lai_max:float = None, # Value indicating the maximum leaf area index of the stand (m2/m2)
                            latitude:Coordinate = None, # Value indicating the latitude of the stand
                            longitude:Coordinate = None, # Value indicating the longitude of the stand
                            sep: str = ";",  # CSV file separator can be ',' or ';'
) -> Dict:  # Dictionary containing parameters

    "Create a dictionary with stand parameters that can be used as in input in \code{run.SureauR}"

    # Assert parameters ---------------------------------------------------------

    # Make sure the file_path exist or is None
    assert file_path is None or os.path.exists(
        file_path
    ), f"Path: {file_path} not found, check spelling"

    # Make sure that if file_path is None, the other params are provided and
    # are the correct data type
    if file_path is None:
        assert (isinstance(lai_max, float)
                ), "Missing lai_max or incorrect data type. Provide lai_max as a floating point i.e. lai_max = 6.0011"

        assert(isinstance(latitude, float) and isinstance(longitude, float)
               ), "Missing latitude and/or longitude. Provide latitude and/or longitude as Coordinates points i.e. latitude = 41.40338, longitude = 2.17403"


    # Raise error if file_path and parameters are provided
    if file_path is not None:
        assert (lai_max is None and latitude is None and longitude is None
                ),"file_path, lai_max, latitude and longitude were provided. If CSV file is provided set lai_max, latitude and longitude to None"

    # Create stand_parameters from csv file -------------------------------------
    if file_path is not None:

        # Read file
        stand_params_csv = pd.read_csv(file_path, header=0, sep=sep)

        # Raise error if stand_params_csv don't follow the StandFile Schema
        StandFile.validate(stand_params_csv)

        # Convert a plot_id column to rownames
        stand_params_csv = stand_params_csv.set_index('plot_id')

        # Covert dataframe to dictionary
        stand_params = stand_params_csv.to_dict('index')

        # Return dictionary as a defaultdict to keep consistency
        return collections.defaultdict(list, stand_params)

    # Create stand_parameters from function inputs ------------------------------

    # Create empty dictionary
    stand_params = collections.defaultdict(list)

    if file_path is None:
        stand_params["lai_max"] = lai_max
        stand_params["latitude"] = latitude
        stand_params["longitude"] = longitude

        return stand_params


## Create stand parameters 

In [161]:
create_stand_parameters(file_path="./sample_data/stand_example.csv",
                        lai_max=None,
                        latitude=None,
                        longitude=None,
                        sep=";"
                        )

TypeError: dtype '<class 'pydantic_extra_types.coordinate.Coordinate'>' not understood

In [159]:
create_stand_parameters(file_path = None,
                        lai_max=2.0,
                        latitude=41.1,
                        longitude=2.1
                        )

defaultdict(list, {'lai_max': 2.0, 'latitude': 41.1, 'longitude': 2.1})